In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np
import torch.nn.functional as F

from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding
from torch.utils.data import Dataset

experiment_name = 'InSlateAttentionSequencewiseGRU'
device = 'cuda:2'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
# torch.autograd.set_detect_anomaly(True)

class AttentionResponseModel(torch.nn.Module):
    """
    No recurrent dependency, just slate-wise attention.
    """
    def __init__(self, embedding, nheads=2, output_dim=1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.attention = torch.nn.MultiheadAttention(
            2 * embedding.embedding_dim,
            num_heads=nheads,
            batch_first=True
        )
        
        self.rnn_layer = torch.nn.GRU(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
            batch_first=True
        )
        
        self.out_layer = torch.nn.Linear(3 * embedding.embedding_dim, output_dim)
    
    
    def get_attention_embeddings(self, item_embs, user_embs, slate_mask):
        shp = item_embs.shape      
        key_padding_mask = slate_mask
        key_padding_mask[:,:, 0] = True # let model attent to first padd token if slate is empty 
        features = torch.cat(
            [
                item_embs,
                user_embs[:, :, None, :].repeat(1, 1, item_embs.size(-2), 1).reshape(shp)
            ],
            dim = -1
        ).flatten(0,1)
        
        features, attn_map = self.attention(
            features, features, features,
            key_padding_mask=~key_padding_mask.flatten(0, 1)
        )
        shp = list(shp)
        shp[-1] *= 2
        features = features.reshape(shp)
        return features
    
    def forward(self, batch):
        # consider sequential clicks, hence need to flatten slates
        item_embs, user_embs = self.embedding(batch)
        slate_mask = batch['slates_mask'].clone()
        
        att_features = self.get_attention_embeddings(item_embs, user_embs, slate_mask)
        
        gru_features, _ = self.rnn_layer(item_embs.flatten(-3, -2))
        gru_features = gru_features.reshape(item_embs.shape)
        
        features = torch.cat(
            [att_features, gru_features],
            dim=-1
        )
        
        return self.out_layer(features).squeeze(-1)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = AttentionResponseModel(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 2561.15it/s]


Test before learning: {'f1': 0.0, 'roc-auc': 0.6666666269302368, 'accuracy': 0.75}


train... loss:0.6311895251274109:   0%|                                                                                                    | 1/5000 [00:00<26:25,  3.15it/s]

Val update: epoch: 0 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 0.6666666269302368 | treshold: 0.45
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 0.6666666269302368 | 


train... loss:0.5611900091171265:   0%|                                                                                                    | 4/5000 [00:01<22:31,  3.70it/s]

Val update: epoch: 3 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.32
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 


train... loss:0.541406512260437:   0%|                                                                                                     | 5/5000 [00:01<26:38,  3.12it/s]

Val update: epoch: 5 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.55
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(AttentionResponseModel(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (attention): MultiheadAttention(
     (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
   )
   (rnn_layer): GRU(32, 32, batch_first=True)
   (out_layer): Linear(in_features=96, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = AttentionResponseModel(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating InSlateAttentionSequencewiseGRU with svd embeddings
Test before learning: {'f1': 0.183157280087471, 'roc-auc': 0.52588951587677, 'accuracy': 0.10081075131893158}


train... loss:34.88440942764282:   0%|                                                                                                  | 2/5000 [01:17<50:19:25, 36.25s/it]

Val update: epoch: 1 |accuracy: 0.09848575294017792 | f1: 0.1793118417263031 | auc: 0.5717933177947998 | treshold: 0.03
Test: accuracy: 0.10081075131893158 | f1: 0.183157280087471 | auc: 0.5568777322769165 | 


train... loss:33.90444603562355:   0%|                                                                                                  | 3/5000 [01:41<42:35:36, 30.69s/it]

Val update: epoch: 2 |accuracy: 0.13060390949249268 | f1: 0.18345721065998077 | auc: 0.6018333435058594 | treshold: 0.05
Test: accuracy: 0.13061539828777313 | f1: 0.18755140900611877 | auc: 0.588494598865509 | 


...

train... loss:28.8349107503891:   0%|▎                                                                                                 | 15/5000 [06:32<35:58:45, 25.98s/it]

Val update: epoch: 14 |accuracy: 0.8668475151062012 | f1: 0.31686803698539734 | auc: 0.7442652583122253 | treshold: 0.18000000000000002
Test: accuracy: 0.8627750873565674 | f1: 0.3009754419326782 | auc: 0.7350354194641113 | 


train... loss:28.39010801911354:   0%|▎                                                                                                | 16/5000 [06:58<36:15:00, 26.18s/it]

Val update: epoch: 15 |accuracy: 0.8675926923751831 | f1: 0.3205873370170593 | auc: 0.7491629123687744 | treshold: 0.19
Test: accuracy: 0.8632538318634033 | f1: 0.30933624505996704 | auc: 0.7377991676330566 | 


train... loss:25.918310806155205:   1%|█▎                                                                                              | 66/5000 [28:31<35:32:21, 25.93s/it]


In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['neural','explicit', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = AttentionResponseModel(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating InSlateAttentionSequencewiseGRU with neural embeddings
Test before learning: {'f1': 0.7070940136909485, 'roc-auc': 0.5168448090553284, 'accuracy': 0.5949462056159973}


train... loss:58.82067450881004:   0%|                                                                                                  | 1/5000 [01:03<88:29:15, 63.72s/it]

Val update: epoch: 0 |accuracy: 0.7771247625350952 | f1: 0.840562641620636 | auc: 0.861542820930481 | treshold: 0.4
Test: accuracy: 0.7778019309043884 | f1: 0.8418716192245483 | auc: 0.8600804805755615 | 


train... loss:49.33029007911682:   0%|                                                                                                  | 2/5000 [01:11<42:29:25, 30.61s/it]

Val update: epoch: 1 |accuracy: 0.8133314251899719 | f1: 0.8602874875068665 | auc: 0.8926573395729065 | treshold: 0.45
Test: accuracy: 0.8155966401100159 | f1: 0.8628565073013306 | auc: 0.891123354434967 | 


...

train... loss:41.24025613069534:   1%|█                                                                                                 | 54/5000 [06:05<9:13:16,  6.71s/it]

Val update: epoch: 53 |accuracy: 0.841266393661499 | f1: 0.8737034797668457 | auc: 0.916419506072998 | treshold: 0.47000000000000003
Test: accuracy: 0.8391246795654297 | f1: 0.8726430535316467 | auc: 0.9159804582595825 | 


/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
train... loss:41.020749509334564:   1%|█                                                                                                | 55/5000 [06:12<9:22:05,  6.82s/it]

Val update: epoch: 54 |accuracy: 0.8394040465354919 | f1: 0.8711230158805847 | auc: 0.9173226952552795 | treshold: 0.47000000000000003
Test: accuracy: 0.8387861251831055 | f1: 0.8711815476417542 | auc: 0.9170501828193665 | 


train... loss:41.69330483675003:   1%|█▏                                                                                                | 62/5000 [07:04<9:23:18,  6.84s/it]


In [10]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items